In [ ]:
from lxml import etree
from urllib.request import urlopen
from collections import defaultdict
import itertools
from datetime import timedelta
from datetime import datetime
from gurobipy import *
from random import randrange
import random
from gurobipy import *
import time
import pandas as pd

url = urlopen('https://raw.githubusercontent.com/ArthurK-GH/Instances/main/sprint01.xml')
doc = etree.parse(url)
root = doc.getroot()

In [ ]:
getStart_date = root[0].text
getEnd_date = root[1].text

start_date = datetime.strptime(getStart_date, '%Y-%m-%d')
end_date = datetime.strptime(getEnd_date, '%Y-%m-%d')


delta = (end_date - start_date) #find the delta in number of days (28)

#[datetime.dates]
list_of_dates = []
for i in range(delta.days +1):
    day = start_date + timedelta(days=i)
    list_of_dates.append(day)

#[1,2,3,4,...,n] -> numbering dates from 1 -> cf. input D
delta = (end_date - start_date).days +1
list_of_days = []
for x in range(delta):
    list_of_days.append(x+1)


#Create a dictionary with the dates as key and the days of the planning horizon sequentially numbered 
#from 1 as value. This dictionary will be usefull in the future for the input data
zipbObj = zip(list_of_dates,list_of_days)
DicDatesDays = dict(zipbObj)

In [ ]:
shift_ID = []
shift_start = []
shift_end = []
shift_description = []
shift_skill = [] 

for shiftTypes_data in root.xpath('//ShiftTypes'): 
    shift_ID = shiftTypes_data.xpath('./Shift/@ID') # '@' to find the attribut of the element 'Shift' in this tree
    
    for shift_data in shiftTypes_data.xpath('.//StartTime'):
        shift_start.append(shift_data.text)
        
    for shift_data in shiftTypes_data.xpath('.//EndTime'):
        shift_end.append(shift_data.text)
        
    for shift_data in shiftTypes_data.xpath('.//Description'):
        shift_description.append(shift_data.text)
        
    for shift_data in shiftTypes_data.xpath('.//Skill'):
        shift_skill.append(shift_data.text)
        


In [ ]:
pattern_ID = []
pattern_weight = []
petternEntry_index = []
pettern_shitType = []
pattern_Day = []

for pattern in root.xpath('//Patterns'):
    pattern_ID = pattern.xpath('./Pattern/@ID')
    pattern_ID = [int(p) for p in pattern_ID]
    pattern_weight = pattern.xpath('./Pattern/@weight')
    pattern_weight = [int(p) for p in pattern_weight]
    
    for patternEntries in pattern.xpath('.//PatternEntries'):
        petternEntry_index = patternEntries.xpath('./PatternEntry/@index')
    
        for patternEntry in patternEntries.xpath('.//ShiftType'):
            pettern_shitType.append(patternEntry.text)
        
        for patternEntry in patternEntries.xpath('.//Day'):
            pattern_Day.append(patternEntry.text)

zipbObj = zip(pattern_ID,pattern_weight)
DictPatternID_Weight = dict(zipbObj)

In [ ]:
employee_ID = []
contract_ID = []
employee_skill = []

for employees in root.xpath('//Employees'):
    employee_ID = employees.xpath('./Employee/@ID')
    
    for employee in employees.xpath('./Employee/ContractID'):
        contract_ID.append(employee.text)
        
    for skill in employees.xpath('./Employee/Skills/Skill'):
        employee_skill.append(skill.text)
        
        

employee_ID = [int(i) for i in employee_ID]
Dict_NurseID_ContractID = dict(zip(employee_ID,contract_ID))

In [ ]:
DictDayShiftNbNurses = {}
current_day = None
current_shift = None

#Instead of xpath I used a conditional function in order to create this dictionary.
for element in root.iter():
    if element.tag == "Day":
        current_day = element.text
    elif element.tag == "Shift":
        current_shift = element.text
    elif element.tag == "Preferred":
        DictDayShiftNbNurses[(current_day, current_shift)] = element.text

In [ ]:
dayOff_weight = []
dayOff_employeeID = []
dayOff_date = []
DayOff_Datetime = []
DayOff_DaysD = []

for dayOffRequest in root.xpath('//DayOffRequests'):
    dayOff_weight = dayOffRequest.xpath('./DayOff/@weight')
    dayOff_weight = [int(p) for p in dayOff_weight]
    
    for employeeID in dayOffRequest.xpath('.//EmployeeID'):
        dayOff_employeeID.append(employeeID.text)
    
    for date in dayOffRequest.xpath('.//Date'):
        dayOff_date.append(date.text)

dayOff_employeeID = [int(i) for i in dayOff_employeeID]

#Replace the day off dates by the input D
#1 Put the value in the right format (datetime)
for element in dayOff_date:
    DayOff_Datetime.append(datetime.strptime(element,'%Y-%m-%d'))
    
#2 Match the corresponding value and create the new list
for ID, datetime in zip(dayOff_employeeID,DayOff_Datetime):
    for (datetime2, days) in DicDatesDays.items():
        if datetime == datetime2:
            DayOff_DaysD.append(days)
            
DayOff_DaysD = list(-x for x in DayOff_DaysD)
ListDay_Weight = list(zip(DayOff_DaysD,dayOff_weight))

ID_Dayoff_weight = defaultdict(list)
for ID, date in zip(dayOff_employeeID,ListDay_Weight):
    ID_Dayoff_weight[ID].append(date)
ID_Dayoff_weight = dict(ID_Dayoff_weight)

In [ ]:
shiftOff_weight = []
shiftOff_typeID = []
shiftoff_employeeID = []
shiftOff_date = []
ShiftOff_Datetime = []
ShiftOff_DaysD = []

for shiftOffRequests in root.xpath('//ShiftOffRequests'):
    shiftOff_weight = (shiftOffRequests.xpath('./ShiftOff/@weight'))
    shiftOff_weight = [int(p) for p in shiftOff_weight]
    
    for shitTypeID in shiftOffRequests.xpath('.//ShiftTypeID'):
        shiftOff_typeID.append(shitTypeID.text)
        
    for employeeID in shiftOffRequests.xpath('.//EmployeeID'):
        shiftoff_employeeID.append(employeeID.text)
    
    for date in shiftOffRequests.xpath('.//Date'):
        shiftOff_date.append(date.text)

shiftoff_employeeID = [int(i) for i in shiftoff_employeeID]

#Replace the shift off dates by the days D
#1 Put the value in the right format (datetime)
for element in shiftOff_date:
    ShiftOff_Datetime.append(datetime.strptime(element,'%Y-%m-%d'))
    
#2 Match the corresponding value and create the new list
for ID, datetime in zip(shiftoff_employeeID,ShiftOff_Datetime):
    for (datetime2, days) in DicDatesDays.items():
        if datetime == datetime2:
            ShiftOff_DaysD.append(days)

# 1 Input

## 1.1 Input Data

$N$ is a set of nurses, with $N = \{0, 1, ..., n\}$

$S$ is a set of shift, with $S = \{E, L, D, N\}$ ; $Sd = \{E, L, D\}$ (day shift) and  $Sn = \{N\}$ (night sift)

$D$ is a set of days, with $D = \{1, 2, ..., d\}$ : sequentially numbered from 1

$R$ is the number of required nurses at day $d$ ans shift $s$ 

$P$ is a set of all ordered pair, with $P = \{(1,2),(2,3),..., p\}$ representing windows in the planning horizon
 
$Wn$ is a set of weekend, with $W = \{1, 2, ..., wn\}$ in the planning horizon for nurse $n$
     
$Din$ is a set of days in the $i$-th weekend of nurse $n$

$Lin$ is the last day before the $i$-th weekend of nurse $n$

$UShift$ is a set of unwanted working shift patterns for nurse $n$

$UDay$ is a set of unwanted working days patterns for nurse $n$

In [ ]:
#goal of the block: input N,S{(Sd),(Sn)}, D
N = employee_ID
S = shift_ID
Sd = []
Sn = []
for x in S:
    if x == "N":
        Sn.append(x)
    else:
        Sd.append(x)
        
D = list_of_days
D_neg = list(-x for x in D)

In [ ]:
#For unwanted day patterns
D_NotWorking = list(-x for x in D)

In [ ]:
#goal of the block: input R {([DaysD],[Shift]:[CoverRequirement])}
ListD = [] 
ListShift = []
ListNbNurses = []


for (key, value) in DicDatesDays.items():
    if key.weekday() == 0:  #0 for Monday
        for (day, shift), nb in DictDayShiftNbNurses.items():
            if day == 'Monday':
                ListD.append(value)
                ListShift.append(shift)
                ListNbNurses.append(nb)
                
    if key.weekday()== 1: #1 for Tuesday
        for (day, shift), nb in DictDayShiftNbNurses.items():
            if day == 'Tuesday': 
                ListD.append(value)
                ListShift.append(shift)
                ListNbNurses.append(nb)
                
    if key.weekday() == 2:
        for (day, shift), nb in DictDayShiftNbNurses.items():
            if day == 'Wednesday':
                ListD.append(value)
                ListShift.append(shift)
                ListNbNurses.append(nb)
                
    if key.weekday() == 3:
        for (day, shift), nb in DictDayShiftNbNurses.items():
            if day == 'Thursday':
                ListD.append(value)
                ListShift.append(shift)
                ListNbNurses.append(nb)
                
    if key.weekday() == 4:
        for (day, shift), nb in DictDayShiftNbNurses.items():
            if day == 'Friday':
                ListD.append(value)
                ListShift.append(shift)
                ListNbNurses.append(nb)
                
    if key.weekday() == 5:
        for (day, shift), nb in DictDayShiftNbNurses.items():
            if day == 'Saturday':
                ListD.append(value)
                ListShift.append(shift)
                ListNbNurses.append(nb)
                
    if key.weekday() == 6:
        for (day, shift), nb in DictDayShiftNbNurses.items():
            if day == 'Sunday':
                ListD.append(value)
                ListShift.append(shift)
                ListNbNurses.append(nb)

ListNbNurses = [int(i) for i in ListNbNurses]

#Now I just need to zip all 3 lists in a dictionary and since they have the same lenght I can use "for/in/len"          
R = {(ListD[i],ListShift[i]):ListNbNurses[i] for i in range(len(ListD))}

In [ ]:
#goal of the block input P
P1 = list(itertools.product(D, repeat=2))
P = []
for d1,d2 in P1:
    if d1 <= d2:
        P += [(d1,d2)]

In [ ]:
#Goal of the block input W
#1 Create a dictionary to get the contract ID as key and the weekend definition as value {ContractID: WEDefintion}
ContractID_DictWE = []
WeekendDefinition = []

for ID in root.xpath('//Contracts'):
    ContractID_DictWE = ID.xpath('./Contract/@ID')
    for WEDef in root.xpath('.//Contract/WeekendDefinition'):
        WeekendDefinition.append(WEDef.text)
        
DictWE = dict(zip(ContractID_DictWE,WeekendDefinition))

#2 Dictionary {Weekeend:[dates]}
list_1 = []
list_2 = []

#2.1 From Saturday to Sunday (weekend definition)
#2.11 List of days D (input D) in the weekend
for i in range(delta + 1):
    day = start_date + timedelta(days=i)
    if day.weekday() >= 5: # >= Saturday
        list_1.append(day)
for dates, day in DicDatesDays.items():
    for dates2 in list_1:
        if dates == dates2:
            list_2.append(day)

#2.2 DictSatToSun = {WE = [dates]}
DictSatToSun = {}

#Keep empty dict unless there are items.
if len(list_2) > 0:
    # Populate first keyed item with first element.
    key = 1
    DictSatToSun = {key:[list_2[0]]}

    # Process other elements.
    for num in list_2[1:]:
    # Consecutive? Add to existing. Otherwise, add to new key.
        
        if num == DictSatToSun[key][-1] + 1:
            DictSatToSun[key].append(num)
        else:
            key += 1
            DictSatToSun[key] = [num]
            
l = []
for k, v in DictSatToSun.items():
    l.append(v)

SatToSun = list(map(tuple,l)) #convert into tuples [(dates)]

list_1 = []
list_2 = []

#2.3 Friday to Sunday (weekend definition)
for i in range(delta):
    day = start_date + timedelta(days=i)
    if day.weekday() >= 4: #>= Friday
        list_1.append(day)
for dates, day in DicDatesDays.items():
    for dates2 in list_1:
        if dates == dates2:
            list_2.append(day)
            
#2.4 DictFryToSun = {WE = [dates]}
DictFryToSun = {}
if len(list_2) > 0:
    key = 1
    DictFryToSun = {key:[list_2[0]]}
    
    for num in list_2[1:]:
        if num == DictFryToSun[key][-1] + 1:
            DictFryToSun[key].append(num)
        else:
            key += 1
            DictFryToSun[key] = [num]
            
l = []
for k, v in DictFryToSun.items():
    l.append(v)

FryToSun = list(map(tuple,l)) #convert into tuples [(dates)]

#3 Dict1 = {ContractID:[(dates)]}
Dict1 = {}
for (ContractID, WEdef) in DictWE.items():
    if WEdef == 'SaturdaySunday':
        Dict1[ContractID] = SatToSun
    if WEdef == 'FridaySaturdaySunday':
        Dict1[ContractID] = FryToSun

#4 W = {NurseID:[(dates)]
Wn_d = {}
for nurse_ID, Contract_ID in Dict_NurseID_ContractID.items():
    for Contract_ID2, dates in Dict1.items():
        if Contract_ID == Contract_ID2:
            Wn_d[nurse_ID] = dates

In [ ]:
#goal of the block: index I, index N_I & input Din
unique_WE = sorted(set(itertools.chain.from_iterable(Wn_d.values())))
map_I_WE = {k+1: v for k, v in enumerate(unique_WE)} #+1 so elements start at one and not 0
map_WE_I = {map_I_WE[k]: k for k in map_I_WE}

I = list(map_I_WE.keys())

Wn = defaultdict(list)
for n in Wn_d:
    for w in Wn_d[n]:
        Wn[n].append(map_WE_I[w])
        
Wn = dict(Wn)       
Din = {}
for k1, v1 in Wn_d.items():
    for k2, v2 in map_I_WE.items():
        for element in v1:
            if element == v2:
                 Din[(k2,k1)] = v2

In [ ]:
#goal of the block: input Lin
Dict_InputLin = dict(zip(ContractID_DictWE,WeekendDefinition))

List_lastDayFri = []
List_lastDayThu = []

for (key, value) in DicDatesDays.items():
    if key.weekday() == 4:
        List_lastDayFri.append(value)
        
for (key, value) in DicDatesDays.items():
    if key.weekday() == 3:
        List_lastDayThu.append(value)
        
for (ContratID, WE) in Dict_InputLin.items():
    if WE == 'SaturdaySunday':
        Dict_InputLin[ContratID] = List_lastDayFri
    elif WE == 'FridaySaturdaySunday':
        Dict_InputLin[ContratID] = List_lastDayThu  #Planning horizon starting a Friday (Sprint_late09) -> Issue 1st WE nurse 9 ?

        
DictN_SetLastDayWE = {}
for (Nurse_ID, ContractID) in Dict_NurseID_ContractID.items():
    for (ContractID2, Days) in Dict_InputLin.items():
        if ContractID == ContractID2:
            DictN_SetLastDayWE[Nurse_ID] = Days
            
UniqueLastDayWE = sorted(set(itertools.chain.from_iterable(DictN_SetLastDayWE.values())))
map_I_LastDayWE = {k+1: v for k, v in enumerate(UniqueLastDayWE)}

Lin = {}
for k,v in DictN_SetLastDayWE.items():
    for k2, v2 in map_I_LastDayWE.items():
        for element in v:
            if element == v2:
                Lin[k2,k] = v2

In [ ]:
#Goal of the block: formating data for the input UShift & UDays by extracting contract related information

#Dictionary: {Contract_ID:[UnwantedPatterns]}
COntractID_UnwantedPatt = {} 
cont = doc.xpath('//Contract')
for element in cont:
    key = element.xpath('./@ID')[0]
    ListUnwantedPatternID = element.xpath('.//UnwantedPatterns/Pattern/text()')
    ListUnwantedPatternID = [int(p) for p in ListUnwantedPatternID]
    COntractID_UnwantedPatt[key] = ListUnwantedPatternID


#Dictionary: {Nurse_ID:[UnwantedPatterns]}
DictID_UP = {}
for (nurseID, ContractID) in Dict_NurseID_ContractID.items():
    for (ContractID2, UP) in COntractID_UnwantedPatt.items():
        if ContractID == ContractID2:
            DictID_UP[nurseID] = UP

#Dictionary: {UnwantedPattern:[(shifts:days)]

DictShift_Day = {}
pat = root.xpath('//Patterns/Pattern')
for element in pat:
    key = element.xpath('./@ID')[0]  #[0] important so xpath doesn't return a list
    shift_Day = list(zip(element.xpath('.//ShiftType/text()'),element.xpath('.//Day/text()')))
    DictShift_Day[key] = shift_Day

DictShift_Day = {int(k):v for k,v in DictShift_Day.items()}

#Goal: identify shifts related patterns and days related

DictUP_Days = {}
DictUP_Shifts = {}

for ID, list_1 in DictShift_Day.items():
    for shift , day in list_1:
        if day == 'Friday': 
            DictUP_Days[ID] = list_1
        if day == 'Any':
            DictUP_Shifts[ID] = list_1        


In [ ]:
##Goal of the block: Ushifts

#Delete 'Any' from the list of unwanted shift
for key, value in DictUP_Shifts.items():
    for index, (shift, days) in enumerate(value):
            if days == 'Any':
                value[index] = shift
                
UShift_ID =[]
UShift = {}
for k1, v1 in DictUP_Shifts.items():
    for k2,v2 in DictID_UP.items():
        for element in v2:
            if element == k1:
                UShift[(k2,k1)] = v1
                UShift_ID.append(k1)
                
#List of pattern ID in Ushift              
UShift_ID = [x for i,x in enumerate(UShift_ID) if x not in UShift_ID[i+1:]] #delete duplicates

#ID_Shift : weight
W11nu ={}
for k1 ,v1 in UShift.items():
    for k2, v2 in DictPatternID_Weight.items():
        if k1[1] == k2:
            W11nu[k1] = v2

In [ ]:
#Formating data for the unwanted pattern: not working on friday and working on the WE
list_Friday_Datetime = []
list_Saturday_Datetime = []
list_Sunday_Datetime = []
list_Friday = []
list_Saturday = []
list_Sunday = []

#Find in my list of datetime all friday/Sat/Sun
for i in range(delta):
    day = start_date + timedelta(days=i)
    if day.weekday() == 4: # = Friday
        list_Friday_Datetime.append(day)
    if day.weekday() == 5: # = Saturday
        list_Saturday_Datetime.append(day)
    if day.weekday() == 6: # = Sunday
        list_Sunday_Datetime.append(day)
        
#Convert datetime into D
for dates, day in DicDatesDays.items():
    for dates2 in list_Friday_Datetime:
        if dates == dates2:
            list_Friday.append(day)
            
for dates, day in DicDatesDays.items():
    for dates2 in list_Saturday_Datetime:
        if dates == dates2:
            list_Saturday.append(day)

for dates, day in DicDatesDays.items():
    for dates2 in list_Sunday_Datetime:
        if dates == dates2:
            list_Sunday.append(day)
            

In [ ]:
#Goal: {NurseID:[Unwated_Days_Pattern],[DaysOffRequests]}

list_Friday = list(-x for x in list_Friday) #list of negative days who represent the none working days
for key, value in DictUP_Days.items():
    for index, (shift, day) in enumerate(value):
            if day == 'Friday': 
                value[index] = list_Friday
            elif day == 'Saturday':
                value[index] = list_Saturday
            elif day == 'Sunday':
                value[index] = list_Sunday
                
                
#Group the following days together and create a list of unwanted days pattern for the model
List_UnwantedDaysPattern = []
for k, v in DictUP_Days.items():
    v1 = list(zip(*v))
    DictUP_Days[k]=v1
    List_UnwantedDaysPattern = v1
    
Day_ID = []
NurseID_UPattern = {}
for k1, v1 in DictUP_Days.items():
    for k2,v2 in DictID_UP.items():
        for element in v2:
            if element == k1:
                NurseID_UPattern[k2] = v1
                UDay_ID2 = k1 #Pattern ID for days of
                
                
ID_Dayoff = defaultdict(list)
for ID, date in zip(dayOff_employeeID,DayOff_DaysD):
    ID_Dayoff[ID].append(date)
    
for k,v in ID_Dayoff.items():
    ID_Dayoff[k] = [(x,) for x in v]

UDay_n = defaultdict(list)
for patterns in (NurseID_UPattern,ID_Dayoff):
    for key, value in patterns.items():
        UDay_n[key].extend(value)
        
UDay_n = dict(UDay_n)



In [ ]:
dayOff_weight = [int(i) for i in dayOff_weight]

#{NurseID : [weight]} for the days off requests
W12n = defaultdict(list)
for ID, weight in zip(dayOff_employeeID,dayOff_weight):
    W12n[ID].append(weight)
    
for k,v in W12n.items():
    W12n[k].append(1)
    
for k,v in W12n.items():
    W12n[k] = sum(v)/len(v)
    
W12n= dict(W12n)

## 1.2 Input Constraints: limits, weights, penalities

### 1.2.1 Ranged soft constraints

    1.Minimum/Maximum number of shifts assigned to a nurse
    2.Minimum/Maximum number of consecutive free days (not in objective function)
    3.Minimum/Maximum number of consecutive working days (not in objective function)
    4.Maximum number of working weekends in four weeks
    5.Minimum/maximum number of consecutive working weekends (not in objective function)
    6.Number of days off after a series of night shifts -> Sould be in the contract, weight associted in the objective function, in the rules of INCR-1 -> 'Two free days after a night shift: Nurses should not be assigned any shift except a night shift during the followingtwo days after a night shift'
    

In [ ]:
#Goal of the block: Input constraint 1

DictMaxAssi = {}
DictMinAssi = {}
DictWeightMax = {}
DictWeightMin = {}

cont = root.xpath('//Contract')
for element in cont:
    key = element.xpath('./@ID')[0]
    ListMaxAssignements = element.xpath('.//MaxNumAssignments/text()')
    ListMaxAssignements = [int(p) for p in ListMaxAssignements] #convert to integers
    ListMaxAssignements = ListMaxAssignements[0] #remove extra brackets
    ListMinAssignements = element.xpath('.//MinNumAssignments/text()')
    ListMinAssignements = [int(p) for p in ListMinAssignements]#convert to integers
    ListMinAssignements = ListMinAssignements[0]#remove extra brackets
    ListMaxWeight = element.xpath('.//MaxNumAssignments/@weight')
    ListMaxWeight = [int(p) for p in ListMaxWeight]#convert to integers
    ListMaxWeight = ListMaxWeight[0]#remove extra brackets
    ListMinWeight = element.xpath('.//MinNumAssignments/@weight')
    ListMinWeight = [int(p) for p in ListMinWeight]#convert to integers
    ListMinWeight = ListMinWeight[0]#remove extra brackets
    DictMaxAssi[key] = ListMaxAssignements
    DictMinAssi[key] = ListMinAssignements
    DictWeightMax[key] = ListMaxWeight
    DictWeightMin[key] = ListMinWeight
    

#Lower limit for nurse N
YL1n = {}

for (nurseID, ContractID) in Dict_NurseID_ContractID.items():
    for (ContractID2, Min) in DictMinAssi.items():
        if ContractID == ContractID2:
            YL1n[nurseID] = Min
            
#Upper limit for nurse N
YU1n = {}

for (nurseID, ContractID) in Dict_NurseID_ContractID.items():
    for (ContractID2, Max) in DictMaxAssi.items():
        if ContractID == ContractID2:
            YU1n[nurseID] = Max
            
#weight lower limit for nurse N
WL1n = {}

for (nurseID, ContractID) in Dict_NurseID_ContractID.items():
    for (ContractID2, Min) in DictWeightMin.items():
        if ContractID == ContractID2:
            WL1n[nurseID] = Min

#weight upper limit for nurse N
WU1n = {}

for (nurseID, ContractID) in Dict_NurseID_ContractID.items():
    for (ContractID2, Max) in DictWeightMax.items():
        if ContractID == ContractID2:
            WU1n[nurseID] = Max

In [ ]:
#Goal of the block: Input Constraint 2
DictMax = {}
DictMin = {}
DictWeightMax = {}
DictWeightMin = {}

cont = root.xpath('//Contract')
for element in cont:
    key = element.xpath('./@ID')[0]
    ListMax = element.xpath('.//MaxConsecutiveFreeDays/text()')
    ListMax = [int(p) for p in ListMax]
    ListMax = ListMax[0]
    ListMin = element.xpath('.//MinConsecutiveFreeDays/text()')
    ListMin = [int(p) for p in ListMin]
    ListMin = ListMin[0]
    ListMaxWeight = element.xpath('.//MaxConsecutiveFreeDays/@weight')
    ListMaxWeight = [int(p) for p in ListMaxWeight]
    ListMaxWeight = ListMaxWeight[0]
    ListMinWeight = element.xpath('.//MinConsecutiveFreeDays/@weight')
    ListMinWeight = [int(p) for p in ListMinWeight]
    ListMinWeight = ListMinWeight[0]
    DictMax[key] = ListMax
    DictMin[key] = ListMin
    DictWeightMax[key] = ListMaxWeight
    DictWeightMin[key] = ListMinWeight
    
    
#Lower limit for nurse N
YL2n = {}

for (nurseID, ContractID) in Dict_NurseID_ContractID.items():
    for (ContractID2, Min) in DictMin.items():
        if ContractID == ContractID2:
            YL2n[nurseID] = Min
            
#Upper limit for nurse N
YU2n = {}

for (nurseID, ContractID) in Dict_NurseID_ContractID.items():
    for (ContractID2, Max) in DictMax.items():
        if ContractID == ContractID2:
            YU2n[nurseID] = Max 
            
#weight lower limit for nurse N
WL2n = {}

for (nurseID, ContractID) in Dict_NurseID_ContractID.items():
    for (ContractID2, Min) in DictWeightMin.items():
        if ContractID == ContractID2:
            WL2n[nurseID] = Min

#weight upper limit for nurse N
WU2n = {} 

for (nurseID, ContractID) in Dict_NurseID_ContractID.items():
    for (ContractID2, Max) in DictWeightMax.items():
        if ContractID == ContractID2:
            WU2n[nurseID] = Max

In [ ]:
#Goal of the block: Input Constraint 3
DictMax = {}
DictMin = {}
DictWeightMax = {}
DictWeightMin = {}

cont = root.xpath('//Contract')
for element in cont:
    key = element.xpath('./@ID')[0]
    ListMax = element.xpath('.//MaxConsecutiveWorkingDays/text()')
    ListMax = [int(p) for p in ListMax]
    ListMax = ListMax[0]
    ListMin = element.xpath('.//MinConsecutiveWorkingDays/text()')
    ListMin = [int(p) for p in ListMin]
    ListMin = ListMin[0]
    ListMaxWeight = element.xpath('.//MaxConsecutiveWorkingDays/@weight')
    ListMaxWeight = [int(p) for p in ListMaxWeight]
    ListMaxWeight = ListMaxWeight[0]
    ListMinWeight = element.xpath('.//MinConsecutiveWorkingDays/@weight')
    ListMinWeight = [int(p) for p in ListMinWeight]
    ListMinWeight = ListMinWeight[0]
    DictMax[key] = ListMax
    DictMin[key] = ListMin
    DictWeightMax[key] = ListMaxWeight
    DictWeightMin[key] = ListMinWeight


#Lower limit for nurse N
YL3n = {}

for (nurseID, ContractID) in Dict_NurseID_ContractID.items():
    for (ContractID2, Min) in DictMin.items():
        if ContractID == ContractID2:
            YL3n[nurseID] = Min
            
#Upper limit for nurse N
YU3n = {}

for (nurseID, ContractID) in Dict_NurseID_ContractID.items():
    for (ContractID2, Max) in DictMax.items():
        if ContractID == ContractID2:
            YU3n[nurseID] = Max
            
#weight lower limit for nurse N
WL3n = {}

for (nurseID, ContractID) in Dict_NurseID_ContractID.items():
    for (ContractID2, Min) in DictWeightMin.items():
        if ContractID == ContractID2:
            WL3n[nurseID] = Min

#weight upper limit for nurse N
WU3n = {}

for (nurseID, ContractID) in Dict_NurseID_ContractID.items():
    for (ContractID2, Max) in DictWeightMax.items():
        if ContractID == ContractID2:
            WU3n[nurseID] = Max

In [ ]:
#Goal of the block: Input Constraint 4
DictMax = {}
DictWeightMax = {}

cont = root.xpath('//Contract')
for element in cont:
    key = element.xpath('./@ID')[0]
    ListMax = element.xpath('.//MaxWorkingWeekendsInFourWeeks/text()')
    ListMax = [int(p) for p in ListMax]
    ListMax = ListMax[0]
    ListMaxWeight = element.xpath('.//MaxWorkingWeekendsInFourWeeks/@weight')
    ListMaxWeight = [int(p) for p in ListMaxWeight]
    ListMaxWeight = ListMaxWeight[0]
    DictMax[key] = ListMax
    DictWeightMax[key] = ListMaxWeight
    
    
#Upper limit for nurse N
YU4n = {}

for (nurseID, ContractID) in Dict_NurseID_ContractID.items():
    for (ContractID2, Max) in DictMax.items():
        if ContractID == ContractID2:
            YU4n[nurseID] = Max


#weight upper limit for nurse N
WU4n = {}

for (nurseID, ContractID) in Dict_NurseID_ContractID.items():
    for (ContractID2, Max) in DictWeightMax.items():
        if ContractID == ContractID2:
            WU4n[nurseID] = Max

In [ ]:
#Goal of the block: Input Constraint 5
DictMax = {}
DictMin = {}
DictWeightMax = {}
DictWeightMin = {}

cont = root.xpath('//Contract')
for element in cont:
    key = element.xpath('./@ID')[0]
    ListMax = element.xpath('.//MaxConsecutiveWorkingWeekends/text()')
    ListMax = [int(p) for p in ListMax]
    ListMax = ListMax[0]
    ListMin = element.xpath('.//MinConsecutiveWorkingWeekends/text()')
    ListMin = [int(p) for p in ListMin]
    ListMin = ListMin[0]
    ListMaxWeight = element.xpath('.//MaxConsecutiveWorkingWeekends/@weight')
    ListMaxWeight = [int(p) for p in ListMaxWeight]
    ListMaxWeight = ListMaxWeight[0]
    ListMinWeight = element.xpath('.//MinConsecutiveWorkingWeekends/@weight')
    ListMinWeight = [int(p) for p in ListMinWeight]
    ListMinWeight = ListMinWeight[0]
    DictMax[key] = ListMax
    DictMin[key] = ListMin
    DictWeightMax[key] = ListMaxWeight
    DictWeightMin[key] = ListMinWeight

#Lower limit for nurse N
YL5n = {} 

for (nurseID, ContractID) in Dict_NurseID_ContractID.items():
    for (ContractID2, Min) in DictMin.items():
        if ContractID == ContractID2:
            YL5n[nurseID] = Min 
            
#Upper limit for nurse N
YU5n = {}

for (nurseID, ContractID) in Dict_NurseID_ContractID.items():
    for (ContractID2, Max) in DictMax.items():
        if ContractID == ContractID2:
            YU5n[nurseID] = Max 
            
#weight lower limit for nurse N
WL5n = {}

for (nurseID, ContractID) in Dict_NurseID_ContractID.items():
    for (ContractID2, Min) in DictWeightMin.items():
        if ContractID == ContractID2:
            WL5n[nurseID] = Min

#weight upper limit for nurse N
WU5n = {}

for (nurseID, ContractID) in Dict_NurseID_ContractID.items():
    for (ContractID2, Max) in DictWeightMax.items():
        if ContractID == ContractID2:
            WU5n[nurseID] = Max

In [2]:
#Goal of the block: Input Constraint 6 -> Set at 2 days and weight = 1
YU6n = {}
for (nurseID, ContractID) in Dict_NurseID_ContractID.items():
    YU6n[nurseID] = 2
    
WU6n = {}    
for (nurseID, ContractID) in Dict_NurseID_ContractID.items():
    WU6n[nurseID] = 1

NameError: name 'Dict_NurseID_ContractID' is not defined

### 1.2.2 logical soft constraints

    7.Complete weekends -> Binary + weight (not in objective function)
    8.No night shift before free weekend -> Binary + weight
    9.Identical shift types during the weekend -> Binary + weight
    10.Alternative skill (not in objective function)
    11.Unwanted patterns
    12.Day on/off request
    13.Shift on/off reques

In [ ]:
#Input Constraint 7
DictMax = {}
DictMaxWeight = {}
Logical7_1 = {}
W7 = {}

cont = root.xpath('//Contract')
for element in cont:
    key = element.xpath('./@ID')[0]
    ListMax = element.xpath('.//CompleteWeekends/text()')
    ListMax = [str(p) for p in ListMax]
    ListMax = ListMax[0]
    ListMaxWeight = element.xpath('.//CompleteWeekends/@weight')
    ListMaxWeight = [int(p) for p in ListMaxWeight]
    ListMaxWeight = ListMaxWeight[0]
    DictMax[key] = ListMax
    DictMaxWeight[key] = ListMaxWeight
           
for (nurseID, ContractID) in Dict_NurseID_ContractID.items():
    for (ContractID2, Max) in DictMax.items():
        if ContractID == ContractID2:
            Logical7_1[nurseID] = Max
            
logical7 = []
for k,v in Logical7_1.items():
    if v == 'true':
        Logical7 = 'true'
    if v == 'false':
        Logical7 = 'false'

for (nurseID, ContractID) in Dict_NurseID_ContractID.items():
    for (ContractID2, Max) in DictMaxWeight.items():
        if ContractID == ContractID2:
            W7[nurseID] = Max

In [ ]:
#Input Constraint 8
DictMax = {}
DictMaxWeight = {}
Logical8_1 = {}
W8 = {}


cont = root.xpath('//Contract')
for element in cont:
    key = element.xpath('./@ID')[0]
    ListMax = element.xpath('.//NoNightShiftBeforeFreeWeekend/text()')
    ListMax = [str(p) for p in ListMax]
    ListMax = ListMax[0]
    ListMaxWeight = element.xpath('.//NoNightShiftBeforeFreeWeekend/@weight')
    ListMaxWeight = [int(p) for p in ListMaxWeight]
    ListMaxWeight = ListMaxWeight[0]
    DictMax[key] = ListMax
    DictMaxWeight[key] = ListMaxWeight
    
    
for (nurseID, ContractID) in Dict_NurseID_ContractID.items():
    for (ContractID2, Max) in DictMax.items():
        if ContractID == ContractID2:
            Logical8_1[nurseID] = Max
            
Logical8 = []
for k,v in Logical8_1.items():
    if v == 'true':
        Logical8 = 'true'
    if v == 'false':
        Logical8 = 'false'

for (nurseID, ContractID) in Dict_NurseID_ContractID.items():
    for (ContractID2, Max) in DictMaxWeight.items():
        if ContractID == ContractID2:
            W8[nurseID] = Max

In [ ]:
#Input Constraint 9
DictMax = {}
DictMaxWeight = {}
Logical9_1 = {}
W9 = {}

cont = root.xpath('//Contract')
for element in cont:
    key = element.xpath('./@ID')[0]
    ListMax = element.xpath('.//IdenticalShiftTypesDuringWeekend/text()')
    ListMax = [str(p) for p in ListMax]
    ListMax = ListMax[0]
    ListMaxWeight = element.xpath('.//IdenticalShiftTypesDuringWeekend/@weight')
    ListMaxWeight = [int(p) for p in ListMaxWeight]
    ListMaxWeight = ListMaxWeight[0]
    DictMax[key] = ListMax
    DictMaxWeight[key] = ListMaxWeight 

for (nurseID, ContractID) in Dict_NurseID_ContractID.items():
    for (ContractID2, Max) in DictMax.items():
        if ContractID == ContractID2:
            Logical9_1[nurseID] = Max
            
Logical9 = []
for k,v in Logical9_1.items():
    if v == 'true':
        Logical9 = 'true'
    if v == 'false':
        Logical9 = 'false'

for (nurseID, ContractID) in Dict_NurseID_ContractID.items():
    for (ContractID2, Max) in DictMaxWeight.items():
        if ContractID == ContractID2:
            W9[nurseID] = Max

## 1.2.3 Pre computed penalties

In [ ]:
#Create a list with all n,d1,d2 in the planning horizon
List_n = []
for n in N:
    for (d1,d2) in P:
        List_n.append(n)
        
P_1 = P[:]*len(N)
List_n_d1d2 = list(zip(List_n,P_1))

List_n_d1d2 = [(a, *b) for a, b in List_n_d1d2] #remove bracket from tuple inside tuple

In [ ]:
#Penalty sigma -> Min/Max consecutive working days

PenaltySigma = {}
for n,d1,d2 in List_n_d1d2:
    for n1,lower_bound in YL3n.items():
        for n2, upper_bound in YU3n.items():
            
            if n == 0 and n1 == 0 and n2 == 0:
                if (d2-d1)>(upper_bound-lower_bound):
                    PenaltySigma[n,d1,d2] = 1
                elif lower_bound > (d2-d1):
                    PenaltySigma[n,d1,d2] = 1
                else:
                    PenaltySigma[n,d1,d2] = 0
                        
            if n == 1 and n1 == 1 and n == 1:
                if (d2-d1)>(upper_bound-lower_bound):
                    PenaltySigma[n,d1,d2] = 1
                elif lower_bound > (d2-d1):
                    PenaltySigma[n,d1,d2] = 1
                else:
                    PenaltySigma[n,d1,d2] = 0
                    
            if n == 2 and n1 == 2 and n2 == 2:
                if (d2-d1)>(upper_bound-lower_bound):
                    PenaltySigma[n,d1,d2] = 1
                elif lower_bound > (d2-d1):
                    PenaltySigma[n,d1,d2] = 1
                else:
                    PenaltySigma[n,d1,d2] = 0
                    
            if n == 3 and n1 == 3 and n2 == 3:
                if (d2-d1)>(upper_bound-lower_bound):
                    PenaltySigma[n,d1,d2] = 1
                elif lower_bound > (d2-d1):
                    PenaltySigma[n,d1,d2] = 1
                else:
                    PenaltySigma[n,d1,d2] = 0
                    
            if n == 4 and n1 == 4 and n2 == 4:
                if (d2-d1)>(upper_bound-lower_bound):
                    PenaltySigma[n,d1,d2] = 1
                elif lower_bound > (d2-d1):
                    PenaltySigma[n,d1,d2] = 1
                else:
                    PenaltySigma[n,d1,d2] = 0
                    
            if n == 5 and n1 == 5 and n2 == 5:
                if (d2-d1)>(upper_bound-lower_bound):
                    PenaltySigma[n,d1,d2] = 1
                elif lower_bound > (d2-d1):
                    PenaltySigma[n,d1,d2] = 1
                else:
                    PenaltySigma[n,d1,d2] = 0
            
            if n == 6 and n1 == 6 and n2 == 6:
                if (d2-d1)>(upper_bound-lower_bound):
                    PenaltySigma[n,d1,d2] = 1
                elif lower_bound > (d2-d1):
                    PenaltySigma[n,d1,d2] = 1
                else:
                    PenaltySigma[n,d1,d2] = 0
                    
            if n == 7 and n1 == 7 and n2 == 7:
                if (d2-d1)>(upper_bound-lower_bound):
                    PenaltySigma[n,d1,d2] = 1
                elif lower_bound > (d2-d1):
                    PenaltySigma[n,d1,d2] = 1
                else:
                    PenaltySigma[n,d1,d2] = 0
                    
            if n == 8 and n1 == 8 and n2 == 8:
                if (d2-d1)>(upper_bound-lower_bound):
                    PenaltySigma[n,d1,d2] = 1
                elif lower_bound > (d2-d1):
                    PenaltySigma[n,d1,d2] = 1
                else:
                    PenaltySigma[n,d1,d2] = 0
                    
            if n == 9 and n1 == 9 and n2 == 9:
                if (d2-d1)>(upper_bound-lower_bound):
                    PenaltySigma[n,d1,d2] = 1
                elif lower_bound > (d2-d1):
                    PenaltySigma[n,d1,d2] = 1
                else:
                    PenaltySigma[n,d1,d2] = 0

In [ ]:
#Penalty sigma -> + Complete weekend

#To do if time: instances where complete weekend = Friday/Sat/Sun according to WE definition

#list where d1 = d2 during the weekend (works only one days of the weekend)
list_CompleteWE_SatSun = [list_Saturday[0], list_Saturday[0]]
list_CompleteWE_SatSun.extend(([list_Sunday[0],list_Sunday[0],list_Saturday[1],list_Saturday[1],
                 list_Sunday[1],list_Sunday[1],list_Saturday[2],list_Saturday[2],
                 list_Sunday[2],list_Sunday[2],list_Saturday[3],list_Saturday[3],
                 list_Sunday[3],list_Sunday[3]
                ]))

List_n1 = []
for n in N:
    for days in range(1,9):
        List_n1.append(n)
        
list_CompleteWE_SatSun_2 = list_CompleteWE_SatSun[:]*len(N)
list_CompleteWE_SatSun_2 = [*zip(list_CompleteWE_SatSun_2[::2], list_CompleteWE_SatSun_2[1::2])]
list_n_Sat_Sun = list(zip(List_n1,list_CompleteWE_SatSun_2))
list_n_Sat_Sun = [(a, *b) for a, b in list_n_Sat_Sun]

for element in list_n_Sat_Sun:
    for k,v in PenaltySigma.items():
        if element == k:
            PenaltySigma[k] = 1

In [ ]:
#Penalty Tau

PenaltyTau = {}
for n,d1,d2 in List_n_d1d2:
    for n1,lower_bound in YL2n.items():
        for n2, upper_bound in YU2n.items():
            if n == 0 and n1 == 0 and n2 == 0:
                if (d2-d1)>(upper_bound-lower_bound):
                    PenaltyTau[n,d1,d2] = 1
                elif lower_bound > (d2-d1):
                    PenaltyTau[n,d1,d2] = 1
                else:
                    PenaltyTau[n,d1,d2] = 0
                    
            if n == 1 and n1 == 1 and n2 == 1:
                if (d2-d1)>(upper_bound-lower_bound):
                    PenaltyTau[n,d1,d2] = 1
                elif lower_bound > (d2-d1):
                    PenaltyTau[n,d1,d2] = 1
                else:
                    PenaltyTau[n,d1,d2] = 0
                    
            if n == 2 and n1 == 2 and n2 == 2:
                if (d2-d1)>(upper_bound-lower_bound):
                    PenaltyTau[n,d1,d2] = 1
                elif lower_bound > (d2-d1):
                    PenaltyTau[n,d1,d2] = 1
                else:
                    PenaltyTau[n,d1,d2] = 0
                    
            if n == 3 and n1 == 3 and n2 == 3:
                if (d2-d1)>(upper_bound-lower_bound):
                    PenaltyTau[n,d1,d2] = 1
                elif lower_bound > (d2-d1):
                    PenaltyTau[n,d1,d2] = 1
                else:
                    PenaltyTau[n,d1,d2] = 0
                    
            if n == 4 and n1 == 4 and n2 == 4:
                if (d2-d1)>(upper_bound-lower_bound):
                    PenaltyTau[n,d1,d2] = 1
                elif lower_bound > (d2-d1):
                    PenaltyTau[n,d1,d2] = 1
                else:
                    PenaltyTau[n,d1,d2] = 0
                    
            if n == 5 and n1 == 5 and n2 == 5:
                if (d2-d1)>(upper_bound-lower_bound):
                    PenaltyTau[n,d1,d2] = 1
                elif lower_bound > (d2-d1):
                    PenaltyTau[n,d1,d2] = 1
                else:
                    PenaltyTau[n,d1,d2] = 0
            
            if n == 6 and n1 == 6 and n2 == 6:
                if (d2-d1)>(upper_bound-lower_bound):
                    PenaltyTau[n,d1,d2] = 1
                elif lower_bound > (d2-d1):
                    PenaltyTau[n,d1,d2] = 1
                else:
                    PenaltyTau[n,d1,d2] = 0
                    
            if n == 7 and n1 == 7 and n2 == 7:
                if (d2-d1)>(upper_bound-lower_bound):
                    PenaltyTau[n,d1,d2] = 1
                elif lower_bound > (d2-d1):
                    PenaltyTau[n,d1,d2] = 1
                else:
                    PenaltyTau[n,d1,d2] = 0
                    
            if n == 8 and n1 == 8 and n2 == 8:
                if (d2-d1)>(upper_bound-lower_bound):
                    PenaltyTau[n,d1,d2] = 1
                elif lower_bound > (d2-d1):
                    PenaltyTau[n,d1,d2] = 1
                else:
                    PenaltyTau[n,d1,d2] = 0
                    
            if n == 9 and n1 == 9 and n2 == 9:
                if (d2-d1)>(upper_bound-lower_bound):
                    PenaltyTau[n,d1,d2] = 1
                elif lower_bound > (d2-d1):
                    PenaltyTau[n,d1,d2] = 1
                else:
                    PenaltyTau[n,d1,d2] = 0

In [ ]:
#Create a list with all n,i1,i2

#Create a list with all n,d1,d2 in the planning horizon
I1I2_1 = list(itertools.product(I, repeat=2))
I1I2 = []
for i1,i2 in I1I2_1:
    if i1 <= i2:
        I1I2 += [(i1,i2)]



List_n = []
for n in N:
    for (i1,i2) in I1I2:
        List_n.append(n)
        
I_1 = I1I2[:]*len(N)

List_n_i1i2 = list(zip(List_n,I_1))
List_n_i1i2 = [(a, *b) for a, b in List_n_i1i2] #remove bracket from tuple inside tuple

In [ ]:
PenaltyPsi = {}
for n,i1,i2 in List_n_i1i2:
    for n1,lower_bound in YL5n.items():
        for n2, upper_bound in YU5n.items():
            if n == 0 and n1 == 0 and n2 == 0:
                if (i2-i1)>(upper_bound-lower_bound):
                    PenaltyPsi[n,i1,i2] = 1
                elif lower_bound > (i2-i1):
                    PenaltyPsi[n,i1,i2] = 1
                else:
                    PenaltyPsi[n,i1,i2] = 0
                    
            if n == 1 and n1 == 1 and n2 == 1:
                if (i2-i1)>(upper_bound-lower_bound):
                    PenaltyPsi[n,i1,i2] = 1
                elif lower_bound > (i2-i1):
                    PenaltyPsi[n,i1,i2] = 1
                else:
                    PenaltyPsi[n,i1,i2] = 0
                    
            if n == 2 and n1 == 2 and n2 == 2:
                if (i2-i1)>(upper_bound-lower_bound):
                    PenaltyPsi[n,i1,i2] = 1
                elif lower_bound > (i2-i1):
                    PenaltyPsi[n,i1,i2] = 1
                else:
                    PenaltyPsi[n,i1,i2] = 0
                    
            if n == 3 and n1 == 3 and n2 == 3:
                if (i2-i1)>(upper_bound-lower_bound):
                    PenaltyPsi[n,i1,i2] = 1
                elif lower_bound > (i2-i1):
                    PenaltyPsi[n,i1,i2] = 1
                else:
                    PenaltyPsi[n,i1,i2] = 0
                    
            if n == 4 and n1 == 4 and n2 == 4:
                if (i2-i1)>(upper_bound-lower_bound):
                    PenaltyPsi[n,i1,i2] = 1
                elif lower_bound > (i2-i1):
                    PenaltyPsi[n,i1,i2] = 1
                else:
                    PenaltyPsi[n,i1,i2] = 0
                    
            if n == 5 and n1 == 5 and n2 == 5:
                if (i2-i1)>(upper_bound-lower_bound):
                    PenaltyPsi[n,i1,i2] = 1
                elif lower_bound > (i2-i1):
                    PenaltyPsi[n,i1,i2] = 1
                else:
                    PenaltyPsi[n,i1,i2] = 0
            
            if n == 6 and n1 == 6 and n2 == 6:
                if (i2-i1)>(upper_bound-lower_bound):
                    PenaltyPsi[n,i1,i2] = 1
                elif lower_bound > (i2-i1):
                    PenaltyPsi[n,i1,i2] = 1
                else:
                    PenaltyPsi[n,i1,i2] = 0
                    
            if n == 7 and n1 == 7 and n2 == 7:
                if (i2-i1)>(upper_bound-lower_bound):
                    PenaltyPsi[n,i1,i2] = 1
                elif lower_bound > (i2-i1):
                    PenaltyPsi[n,i1,i2] = 1
                else:
                    PenaltyPsi[n,i1,i2] = 0
                    
            if n == 8 and n1 == 8 and n2 == 8:
                if (i2-i1)>(upper_bound-lower_bound):
                    PenaltyPsi[n,i1,i2] = 1
                elif lower_bound > (i2-i1):
                    PenaltyPsi[n,i1,i2] = 1
                else:
                    PenaltyPsi[n,i1,i2] = 0
                    
            if n == 9 and n1 == 9 and n2 == 9:
                if (i2-i1)>(upper_bound-lower_bound):
                    PenaltyPsi[n,i1,i2] = 1
                elif lower_bound > (i2-i1):
                    PenaltyPsi[n,i1,i2] = 1
                else:
                    PenaltyPsi[n,i1,i2] = 0

In [ ]:
#Create a list with all [n,s,d] in the planning horizon
List_n = []
for n in N:
    for s in S:
        List_n.append(n)
        
S_1 = S[:]*len(N)
List_n_s = list(zip(List_n,S_1))
List_n_s = List_n_s[:]*len(D)
List_n_s = sorted(List_n_s)

list_d = (D[:]*len(S)*len(N))
List_n_s_d = list(zip(List_n_s,list_d))
List_n_s_d = [(*a, b) for a, b in List_n_s_d]

In [ ]:
#Penalty Nu

#Shift on off request -> Constraint 13 only (there's no alternative skill in sprint instances)
List_nsd_penalty = list(zip(shiftoff_employeeID,shiftOff_typeID,ShiftOff_DaysD))
List_nsd_weight = dict(zip(List_nsd_penalty,shiftOff_weight))

PenaltyNu = {}
for n,s,d in List_n_s_d:
    PenaltyNu[n,s,d] = 0
    for (n1,s1,d1) in List_nsd_penalty:
        if (n1,s1,d1) in List_n_s_d:
            PenaltyNu[(n1,s1,d1)] = 1

In [ ]:
W = []
for w in Wn.values():
    W += w

W = list(set(W))

Din_d1_d2 = []

for (d1,d2) in Din.values():
    Din_d1_d2 += [(d1,d2)]
    
Din_d1_d2 = set(Din_d1_d2)

list_d_inDin = []
for (d1,d2) in Din_d1_d2:
    list_d_inDin.append(d1)
for (d1,d2) in Din_d1_d2:
    list_d_inDin.append(d2)
list_d_inDin = set(list_d_inDin)

N_PID = []
for n in UDay_n:
    for pid in range(len(UDay_n[n])):
        N_PID += [(n, pid)]

In [ ]:
data = {
    #Set
        'N':N,'S':S,'Sn':Sn,'Sd':Sd,'D':D,'P':P,'I':I,'I1I2':I1I2,'UShift_ID':UShift_ID,
    #Parameter   
        'YL1n':YL1n,'YU1n':YU1n,'YL2n':YL2n,'YU2n':YU2n,'YL3n':YL3n,'YU3n':YU3n,'YU4n':YU4n,'YL5n':YL5n,'YU5n':YU5n,
        'YU6n':YU6n,'WL1n':WL1n,'WU1n':WU1n,'WU4n':WU4n,'WU6n':WU6n,'Logical7':Logical7,'Logical8':Logical7,
        'Logical9':Logical9,'W8':W8,'W9':W9,'W11nu':W11nu,'W12n':W12n,
    #Penalties
        'PenaltySigma':PenaltySigma,'PenaltyTau':PenaltyTau,'PenaltyPsi':PenaltyPsi,'PenaltyNu':PenaltyNu,
    #Other dictionaries/list
       'R':R,'list_d_inDin':list_d_inDin,'Wn':Wn,'Lin':Lin,'Din':Din,'UShift':UShift,'UDay_n':UDay_n,'N_PID':N_PID,
    'W':W
       }

# 2 Model

In [ ]:
from pyomo.environ import *
from pyomo.opt import SolverFactory

In [ ]:
model = ConcreteModel()

In [ ]:
#Nurses (n)
model.N = Set(initialize = data['N'])

#Shifts (s)
model.Sn = Set(initialize = data['Sn'])
model.Sd = Set(initialize = data['Sd'])
model.S = model.Sn | model.Sd

#Nurses and days window (d & d1d2 & nd1d2)
model.D = Set(initialize = data['D'])
model.D1D2 = Set(initialize = data['P'])
model.N_D1D2 = Set(within = model.N * model.D1D2,
                  initialize = [(n, d1d2) for n in model.N for d1d2 in model.D1D2])

#days ans shifts (ds)
model.D_S = Set(within = model.D * model.S,
               initialize = [(d, s) for d in model.D for s in model.S])

#nurses shifts days (nsd)
model.N_S_D = Set(within = model.N * model.S * model.D,
                 initialize = [(n, s, d) for n in model.N for s in model.S for d in model.D])


#Set Nurses and weekends (i & ni & in)
model.I = Set(initialize = data['I'])
model.N_I = Set(within=model.N * model.I, 
                initialize=[(n, i) for n in data['Wn'] for i in data['Wn'][n]])
model.I_N = Set(within = model.I * model.N,
               initialize =[(i, n) for i in model.I for n in model.N])


#Nurses and weekends window (i1i2 & ni1i2)
model.I1I2 = Set(initialize = data['I1I2'])
model.N_I1I2 = Set(within = model.N * model.I1I2,
                  initialize = [(n, i1i2) for n in model.N for i1i2 in model.I1I2])


#Unwanted patterns ID (shifts)
model.UShiftID = Set(initialize = data['UShift_ID'])
model.N_UShiftID = Set(within = model.N * model.UShiftID,
                       initialize =  [(n, u_shiftid) for n in model.N for u_shiftid in model.UShiftID])

#nurses, unwanted shift pattern & days (n, u_shiftid, d) -> slack 11
model.N_UShiftID_D = Set(within = model.N * model.UShiftID * model.D,
                 initialize = [(n, u_shiftid,d) for n in model.N for u_shiftid in model.UShiftID for d in model.D])


## 2.0.3 Parameter declaration

In [ ]:
#Ranged soft constraints Lower & Upper limits (index_N)

#1
model.YL1n = Param(model.N,within=NonNegativeIntegers, initialize = data['YL1n'])
model.YU1n = Param(model.N,within=NonNegativeIntegers, initialize = data['YU1n'])
#2
model.YL2n = Param(model.N,within=Any, initialize = data['YL2n'])
model.YU2n = Param(model.N,within=Any, initialize = data['YU2n'])
#3
model.YL3n = Param(model.N,within=Any, initialize = data['YL3n'])
model.YU3n = Param(model.N,within=Any, initialize = data['YU3n'])
#4
model.YU4n = Param(model.N,within=Any, initialize = data['YU4n'])
#5
model.YL5n = Param(model.N,within=Any, initialize = data['YL5n'])
model.YU5n = Param(model.N,within=Any, initialize = data['YU5n'])
#6
model.YU6n = Param(model.N,within=Any, initialize = data['YU6n'])


#1
model.WL1n = Param(model.N,within=Any, initialize = data['WL1n'])
model.WU1n = Param(model.N,within=Any, initialize = data['WU1n'])
#4
model.WU4n = Param(model.N,within=Any, initialize = data['WU4n'])
#6
model.WU6n = Param(model.N,within=Any, initialize = data['WU6n'])


#Logical soft constraints weight (index_N)
#8
model.W8n = Param(model.N,within=Any, initialize = data['W8'])
#9
model.W9n = Param(model.N,within=Any, initialize = data['W9'])
#11
model.W11nu = Param(model.N_UShiftID,within=Any, initialize = data['W11nu'])# Additional Index
#12
model.W12n = Param(model.N , initialize = data['W12n'])

## 2.1 Decision variables

### 2.1.1 Main decision variables

In [ ]:
model.x = Var(model.N_S_D, within=Binary)

### 2.1.2 Auxiliary variables

In [ ]:
model.y = Var(model.N_I, within=Binary)

In [ ]:
model.w = Var(model.N_D1D2, within=Binary)

In [ ]:
model.r = Var(model.N_D1D2, within=Binary)

In [ ]:
model.z = Var(model.N_I1I2, within=Binary)

### 2.1.3 Bounded slack variables

In [ ]:
model.alpha1_L = Var(model.N, within = NonNegativeIntegers)
model.alpha1_U = Var(model.N, within = NonNegativeIntegers)

In [ ]:
model.alpha4 = Var(model.N_I, within = NonNegativeIntegers)

In [ ]:
model.alpha6 = Var(model.N * model.D, within = NonNegativeIntegers)

### 2.1.4 Binary slack variables

In [ ]:
if data['Logical8'] == 'true':
    model.alpha8 = Var(model.N_I, within=Binary)
else:
    model.alpha8 = Param(model.N_I,initialize = 0)

In [ ]:
if data['Logical9'] == 'true':
    model.alpha9 = Var(model.N_D1D2, within=Binary)
else:
    model.alpha9 = Param(model.N_D1D2, initialize = 0)

In [ ]:
model.alpha11 = Var(model.N_UShiftID_D, within=Binary)

In [ ]:
model.alpha12 = Var(data['N_PID'], within=Binary)

## 2.2 Objective function

In [ ]:
def obj_function(model):
     return (
         #1st block -> big sum over model.N
         sum(   
         sum(data['PenaltySigma'][n,d1,d2] * model.w[n,d1,d2] + data['PenaltyTau'][n,d1,d2] * model.r[n,d1,d2] 
             for (d1,d2) in model.D1D2) + 
         sum(data['PenaltyNu'][n,s,d] * model.x[n,s,d] + model.WU1n[n]*model.alpha1_U[n] + model.WL1n[n]*model.alpha1_L[n] 
                 for d in model.D for s in model.S) +
         sum(model.WU4n[n]*model.alpha4[n,i] + model.W8n[n]*model.alpha8[n,i] +
            sum(sum(model.W9n[n]*model.alpha9[n,d1,d2]for d2 in data['Din'][i,n] if d2 > d1) 
                for d1 in data['Din'][i,n])for i in range(1, len(data['I'])+1)) +
         sum(model.WU6n[n] * model.alpha6[n,d] for d in model.D)
        for n in model.N)+
         
         #2nd block -> every lines already sum over model.N
         sum(data['PenaltyPsi'][n,i1,i2] * model.z[n,i1,i2]for n, i_lst in data['Wn'].items() 
             for i1 in i_lst for i2 in i_lst if i2 >= i1)+
         sum(model.W11nu[n,us_id]*model.alpha11[n,us_id,d] for (n, us_id), u_shift in data['UShift'].items()
          for d in range(1, len(model.D) - len(u_shift) +2))+
         sum(model.W12n[n] * model.alpha12[n,p_id] for n in data['UDay_n'] for p_id in range(len(data['UDay_n'][n])))
     )
model.ObjFunction = Objective(rule=obj_function, sense=minimize)

## 2.3 Constraints

In [ ]:
def constraint_1(model, s, d):
    return sum(model.x[n, s, d] for n in model.N) == data['R'][d,s]

model.C1 = Constraint(model.S, model.D, rule = constraint_1)

In [ ]:
def constraint_2(model, n, d):
    return sum(model.x[n,s,d] for s in model.S) <= 1

model.C2 = Constraint(model.N, model.D, rule = constraint_2)

In [ ]:
def constraint_3 (model, n, i, d):
    return model.y[n,i] >= sum(model.x[n,s,d] for s in model.S)
    
model.C3 = Constraint(model.N, model.I,data['list_d_inDin'], rule = constraint_3)

In [ ]:
def constraint_4 (model, n, i):
    return model.y[n,i] <= sum(model.x[n,s,d] for s in model.S for d in data['list_d_inDin'])

model.C4 = Constraint(model.N, model.I, rule = constraint_4)

In [ ]:
def constraint_5 (model, n,d):
    return sum(model.x[n,s,d] for s in model.S) == sum(model.w[n,d1,d2] for (d1,d2) in model.D1D2 
                                                       if d in range(d1,d2+1))

model.C5 = Constraint(model.N, model.D, rule = constraint_5)

In [ ]:
def constraint_6 (model, n, d):
    return sum(model.x[n,s,d] for s in model.S) == 1 - (sum(model.r[n,d1,d2] for (d1,d2) in model.D1D2
                                                            if d in range(d1,d2+1)))

model.C6 = Constraint(model.N, model.D, rule = constraint_6)

In [ ]:
def constraint_7 (model, n, d):
    return sum((model.w[n,d1,d2]+ model.r[n,d1,d2]) for (d1,d2) in model.D1D2 if d in range (d1,d2+1)) == 1

model.C7 = Constraint(model.N, model.D, rule = constraint_7)

In [ ]:
def constraint_8 (model, n, d):
    return sum(model.w[n,d_prime,d] for d_prime in range(1,d+1)) + sum(model.w[n,d+1,d_2prime] 
                                                                     for d_2prime in model.D if d_2prime >= d+1) <= 1

model.C8 = Constraint(model.N, model.D, rule = constraint_8)

In [ ]:
def constraint_9(model, n, d):
    return sum(model.r[n,d_prime,d] for d_prime in range(1,d+1)) + sum(model.r[n,d+1,d_2prime] 
                                                                     for d_2prime in model.D if d_2prime >= d+1) <= 1
model.C9 = Constraint(model.N, model.D, rule = constraint_9)

In [ ]:
def constraint_10_1 (model, n):
    return (
        model.YL1n[n]*model.alpha1_L[n] <= sum(model.x[n,s,d] for s in model.S for d in model.D)
    )

model.C10_1 = Constraint(model.N, rule = constraint_10_1)

In [ ]:
def constraint_10_2 (model, n):
    return (
        sum(model.x[n,s,d] for s in model.S for d in model.D)<=
                 model.YU1n[n]*model.alpha1_U[n]
    )

model.C10_2 = Constraint(model.N, rule = constraint_10_2)

In [ ]:
def constraint_11(model, n, i):
    if n in data['Wn']:
        list_weekends = data['Wn'][n]
        for i in range(1, len(list_weekends) - 2):
            return sum(model.y[n,i_prime] for i_prime in range(i,i+4)) <= model.YU4n[n] + model.alpha4[n,i]

model.C11 = Constraint(model.N, model.I, rule = constraint_11)

In [ ]:
def constraint_12(model, n, d):
    if d <= (len(model.D) - model.YU6n[n]):
        return (
            sum(model.YU6n[n] * model.x[n,s_prime,d] for s_prime in model.Sn) +
            sum(model.x[n,s,d_prime] for s in model.Sd for d_prime in range(d+1,d+model.YU6n[n]+1))
            <= model.YU6n[n] + model.alpha6[n,d])
    else:
        return Constraint.Skip
        
model.C12 = Constraint(model.N, model.D, rule = constraint_12)

In [ ]:
if data['Logical8'] == 'true':
    def constraint_13(model, n, i):
        if (i, n) in data['Lin']:
            d = data['Lin'][i,n]
            return sum(model.x[n,s,d] for s in model.Sn) - model.y[n,i] <= model.alpha8[n,i]
        
    
if Logical8 == 'true':   
    model.C13 = Constraint(model.N, model.I, rule = constraint_13)

In [ ]:
if data['Logical9'] == 'true':
    def constraint_14(model, n, s, i):
        if (i,n) in data['Din']:
            (d1,d2) = data['Din'][i,n]
            return model.alpha9[n,d1,d2] >= model.x[n,s,d1] - model.x[n,s,d2]
        else:
            return Constraint.Skip
        
        
if Logical9 == 'true':
    model.C14 = Constraint(model.N, model.S, model.I, rule = constraint_14)

In [ ]:
if data['Logical9'] == 'true':
    def constraint_15(model, n, s, i):
        if (i,n) in data['Din']:
            (d1,d2) = data['Din'][i,n]
            return model.alpha9[n,d1,d2] >= model.x[n,s,d2] - model.x[n,s,d1]
        else:
            return Constraint.Skip
        
if Logical9 == 'true':
    model.C15 = Constraint(model.N, model.S, model.I, rule = constraint_15)

In [ ]:
def constraint_16(model, n, p, d):
    if (n, p) in data['UShift']:
        shifts = data['UShift'][n, p]
        for d in range(1, len(model.D) - len(shifts) + 1):
            return (sum(model.x[n,s,d+s_id] for s_id, s in enumerate(shifts)) <= 
                    len(shifts) -1 + model.alpha11[n,p,d])
            
model.C16 = Constraint(model.N, model.UShiftID, model.D, rule =constraint_16)

In [ ]:
def contraint_17(model, n, p_id):
    p = data['UDay_n'][n][p_id]
    return (
        sum(model.x[n,s,d] for d in p if d >= 1 for s in model.S) +
        sum(1 - sum(model.x[n, s, -d] for s in model.S) for d in p if d <= -1)
        <= len(p) - 1 + model.alpha12[n,p_id]
    )
        
model.C17 = Constraint(data['N_PID'], rule = contraint_17)

In [3]:
def constraint_18(model, n, i):
    return sum(model.z[n, i_prime, i_2prime] for i_2prime in range(i, len(data['W'])+1) for i_prime in range(1, i+1)) <= 1

model.C18 = Constraint(model.N, data['W'], rule = constraint_18)

NameError: name 'Constraint' is not defined

In [ ]:
def constraint_19(model, n, i):
    return (
        sum(model.z[n,i_prime,i] for i_prime in range(1, i+1)) +
        sum(model.z[n,i+1,i_2prime] for i_2prime in range(i+1, len(data['W'])+1)) <= 1
    )
        
model.C19 = Constraint(model.N, data['W'][:-1], rule = constraint_19)

In [ ]:
def constraint_20(model, n, i):
    return model.y[n,i] == sum(model.z[n,i_prime,i_2prime] 
                                   for i_2prime in range(i, len(data['W'])+1) for i_prime in range(1, i+1))

model.C20 = Constraint(model.N, data['W'], rule = constraint_20)

In [ ]:
SOLVER_NAME = 'gurobi_persistent'
TIME_LIMIT = 4000

opt = SolverFactory(SOLVER_NAME)
opt.set_instance(model)

if SOLVER_NAME == 'cplex':
    opt.options['timelimit'] = TIME_LIMIT
    opt.options['emphasis mip'] = 3
elif SOLVER_NAME == 'glpk':         
    opt.options['tmlim'] = TIME_LIMIT
elif SOLVER_NAME == 'gurobi_persistent':           
    opt.options['TimeLimit'] = TIME_LIMIT
    # see https://www.gurobi.com/documentation/9.1/refman/mip_models.html
    opt.options['MIPFocus'] = 0  # default is 0
    opt.options['Cuts'] = -1  # default is -1
    opt.options['Heuristics'] = 0.05  # default is 0.05
    opt.options['Presolve'] = -1  # default is -1
    
start = time.time()
results = opt.solve(model, tee=True, save_results=False)
end = time.time()
print(results)

In [ ]:
List_valueObj = []
list_time = []
Value_Distance = []

List_valueObj.append(value(model.ObjFunction))
list_time.append(end - start)
Value_Distance.append(0)

# Reoptimization

https://pyomo.readthedocs.io/en/stable/advanced_topics/persistent_solvers.html

In [ ]:
def constraint_preference (model):
    n = random.choice(data['N'])
    s = random.choice(data['S'])
    d = random.choice(data['D'])
    return model.x[n,s,d] == 1


model.CPref_1 = Constraint(rule = constraint_preference)
model.CPref_2 = Constraint(rule = constraint_preference)
model.CPref_3 = Constraint(rule = constraint_preference)
model.CPref_4 = Constraint(rule = constraint_preference)
model.CPref_5 = Constraint(rule = constraint_preference)

#notify the persistent solver to add this constraint to the existent model
opt.add_constraint(model.CPref_1)
opt.add_constraint(model.CPref_2)
opt.add_constraint(model.CPref_3)
opt.add_constraint(model.CPref_4)
opt.add_constraint(model.CPref_5)

In [ ]:
#Add indexed variable to the persistent solver
model.rho_distance = Var(model.N_S_D, within=Binary)

for v in model.rho_distance.values():  
    opt.add_var(v)

In [ ]:
#Weight
WeightDistance = (n * d)/((n*d)+1)

#model.x0[n,s,d]
ModelX_DictExtVal = model.x.extract_values()

In [ ]:
def obj_function(model):
     return (
         #1st block -> big sum over model.N
         sum(   
         sum(data['PenaltySigma'][n,d1,d2] * model.w[n,d1,d2] + data['PenaltyTau'][n,d1,d2] * model.r[n,d1,d2] 
             for (d1,d2) in model.D1D2) + 
         sum(data['PenaltyNu'][n,s,d] * model.x[n,s,d] + model.WU1n[n]*model.alpha1_U[n] + model.WL1n[n]*model.alpha1_L[n] 
                 for d in model.D for s in model.S) +
         sum(model.WU4n[n]*model.alpha4[n,i] + model.W8n[n]*model.alpha8[n,i] +
            sum(sum(model.W9n[n]*model.alpha9[n,d1,d2]for d2 in data['Din'][i,n] if d2 > d1) 
                for d1 in data['Din'][i,n])for i in range(1, len(data['I'])+1)) +
         sum(model.WU6n[n] * model.alpha6[n,d] for d in model.D)
        for n in model.N)+
         
         #2nd block -> every lines already sum over model.N
         sum(data['PenaltyPsi'][n,i1,i2] * model.z[n,i1,i2]for n, i_lst in data['Wn'].items() 
             for i1 in i_lst for i2 in i_lst if i2 >= i1)+
         sum(model.W11nu[n,us_id]*model.alpha11[n,us_id,d] for (n, us_id), u_shift in data['UShift'].items()
          for d in range(1, len(model.D) - len(u_shift) +2))+
         sum(model.W12n[n] * model.alpha12[n,p_id] for n in data['UDay_n'] for p_id in range(len(data['UDay_n'][n])))+
         
         
         #3rd block -> Distance objective
         sum(model.rho_distance[n,s,d] * WeightDistance for n in model.N for s in model.S for d in model.D)
     )
model.ObjFunction = Objective(rule=obj_function, sense=minimize)
opt.set_objective(model.ObjFunction)

In [ ]:
def constraint_distance_1(model, n, s, d):
    return model.rho_distance[n,s,d] >= model.x[n,s,d] - ModelX_DictExtVal[n,s,d]

model.constraint_distance_1 = Constraint(model.N_S_D, rule = constraint_distance_1)

for v in model.constraint_distance_1.values():  
    opt.add_constraint(v)

In [ ]:
def constraint_distance_2(model, n, s, d):
    return model.rho_distance[n,s,d] >= ModelX_DictExtVal[n,s,d] - model.x[n,s,d]

model.constraint_distance_2 = Constraint(model.N_S_D, rule = constraint_distance_2)

for v in model.constraint_distance_2.values():  
    opt.add_constraint(v)

In [ ]:
SOLVER_NAME = 'gurobi_persistent'
TIME_LIMIT = 40000

opt = SolverFactory(SOLVER_NAME)
opt.set_instance(model)

if SOLVER_NAME == 'cplex':
    opt.options['timelimit'] = TIME_LIMIT
    opt.options['emphasis mip'] = 3
elif SOLVER_NAME == 'glpk':         
    opt.options['tmlim'] = TIME_LIMIT
elif SOLVER_NAME == 'gurobi_persistent':           
    opt.options['TimeLimit'] = TIME_LIMIT
    # see https://www.gurobi.com/documentation/9.1/refman/mip_models.html
    opt.options['MIPFocus'] = 0  # default is 0
    opt.options['Cuts'] = -1  # default is -1
    opt.options['Heuristics'] = 0.05  # default is 0.05
    opt.options['Presolve'] = -1  # default is -1
    
start = time.time()
results = opt.solve(model, tee=True, save_results=False)
end = time.time()
print(results)

# Extract results

In [ ]:
#Value distance objective

#Count the number of time the variable model.alpha_distance == 1:
rho_distance = model.rho_distance.extract_values()
count = 0
for k,v in rho_distance.items():
    if v == 1:
        count += 1

Value_Distance.append(count*WeightDistance)

In [ ]:
List_valueObj.append(value(model.ObjFunction))
list_time.append(end - start)

In [ ]:
Dict_df = {'Time with PS':list_time,'Value Obj.':List_valueObj,'Value Distance Obj.':Value_Distance}
df_PS = pd.DataFrame(data=Dict_df)
df_PS

# Reoptimization multiple iterations

In [ ]:
model.del_component(model.CPref_1)
model.del_component(model.CPref_2)
model.del_component(model.CPref_3)
model.del_component(model.CPref_4)
model.del_component(model.CPref_5)

model.CPref_1 = Constraint(rule = constraint_preference)
model.CPref_2 = Constraint(rule = constraint_preference)
model.CPref_3 = Constraint(rule = constraint_preference)
model.CPref_4 = Constraint(rule = constraint_preference)
model.CPref_5 = Constraint(rule = constraint_preference)

opt.add_constraint(model.CPref_1)
opt.add_constraint(model.CPref_2)
opt.add_constraint(model.CPref_3)
opt.add_constraint(model.CPref_4)
opt.add_constraint(model.CPref_5)

In [ ]:
#in the futur Gurobi (License expired 2021-01-16)
SOLVER_NAME = 'gurobi_persistent'
TIME_LIMIT = 40000

opt = SolverFactory(SOLVER_NAME)
opt.set_instance(model)

if SOLVER_NAME == 'cplex':
    opt.options['timelimit'] = TIME_LIMIT
    opt.options['emphasis mip'] = 3
elif SOLVER_NAME == 'glpk':         
    opt.options['tmlim'] = TIME_LIMIT
elif SOLVER_NAME == 'gurobi_persistent':           
    opt.options['TimeLimit'] = TIME_LIMIT
    # see https://www.gurobi.com/documentation/9.1/refman/mip_models.html
    opt.options['MIPFocus'] = 0  # default is 0
    opt.options['Cuts'] = -1  # default is -1
    opt.options['Heuristics'] = 0.05  # default is 0.05
    opt.options['Presolve'] = -1  # default is -1
    
start = time.time()
results = opt.solve(model, tee=True, save_results=False)
end = time.time()
print(results)

In [ ]:
#Value distance objective

#Count the number of time the variable model.alpha_distance == 1:
rho_distance = model.rho_distance.extract_values()
count = 0
for k,v in rho_distance.items():
    if v == 1:
        count += 1

Value_Distance.append(count*WeightDistance)

List_valueObj.append(value(model.ObjFunction))
list_time.append(end - start)

Dict_df = {'Time with PS':list_time,'Value Obj.':List_valueObj,'Value Distance Obj.':Value_Distance}
df_PS = pd.DataFrame(data=Dict_df)
df_PS